In [ ]:
!pip install flask_ngrok
!pip install flask-cors

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
data_dir = "/content/drive/Shareddrives/SWM - NER/models/CRF/"

Mounted at /content/drive


In [ ]:
!pip install sklearn_crfsuite

In [ ]:
from flask import Flask, jsonify
from flask_ngrok import run_with_ngrok
from flask import request
import json
from flask_cors import CORS, cross_origin
import warnings
warnings.filterwarnings('ignore')
import pickle





mapping = { '0': 'O',
			'1': 'B-PER',
			'2': 'I-PER',
			'3': 'B-ORG',
			'4': 'I-ORG',
			'5': 'B-LOC',
			'6': 'I-LOC',
			'7': 'B-MISC',
			'8': 'I-MISC'
		}


def features(sentence, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    return {
        'word': sentence[index],
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'is_capitalized': sentence[index][0].upper() == sentence[index][0],
        'is_all_caps': sentence[index].upper() == sentence[index],
        'is_all_lower': sentence[index].lower() == sentence[index],
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
        'has_hyphen': '-' in sentence[index],
        'is_numeric': sentence[index].isdigit(),
        'capitals_inside': sentence[index][1:].lower() != sentence[index][1:]
    }

model_file = open(data_dir + 'crf', 'rb')     
model = pickle.load(model_file)

model_file.close()

def pos_tag(sentence):
    sentence_features = [features(sentence, index) for index in range(len(sentence))]
    return list(zip(sentence, model.predict([sentence_features])[0]))
 


def pred_wrapper(data_arg):
  data = data_arg['data'].replace("."," ")
  sentences = [i.strip() for i in data.split(" ") if i != '']
  res = pos_tag(sentences)
  ret_list = []
  ner_ctr = 0
  for word,tag in res:
    # print(tag)
    # print(type(tag))
    if tag != '0':
        ret_list.append([word, mapping[tag]])
        ner_ctr+=1
  return json.dumps({'count': ner_ctr, 'data': ret_list})




def web_app():
    app = Flask(__name__)
    run_with_ngrok(app)
    CORS(app)
    @app.route('/', methods=['GET', 'POST'])
    @cross_origin()
    def run_app():
        return "/pred_crf/ for Predictions."

    @app.route('/pred_crf/', methods=['GET', 'POST'])
    def pred_app():
        jsonData = request.get_json(force=True)
        print(jsonData)
        res = pred_wrapper(jsonData)
        return res
    app.run()

web_app()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://c0437a5d5e88.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [06/Apr/2021 02:45:44] "OPTIONS /pred_crf/ HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2021 02:45:44] "POST /pred_crf/ HTTP/1.1" 200 -


{'data': 'Jay Buhner hit his 38th homer and Edgar Martinez 23rd for Seattle. The Orioles remained tied with the White Sox American League'}


...

127.0.0.1 - - [06/Apr/2021 02:59:08] "GET /pred_crf HTTP/1.1" 308 -
127.0.0.1 - - [06/Apr/2021 02:59:08] "GET /pred_crf/ HTTP/1.1" 400 -
127.0.0.1 - - [06/Apr/2021 03:04:09] "OPTIONS /pred_crf/ HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2021 03:04:10] "POST /pred_crf/ HTTP/1.1" 200 -


{'data': 'Jay Buhner hit his 38th homer and Edgar Martinez 23rd for Seattle. The Orioles remained tied with the White Sox American League'}


127.0.0.1 - - [06/Apr/2021 03:04:42] "OPTIONS /pred_crf/ HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2021 03:04:42] "POST /pred_crf/ HTTP/1.1" 200 -


{'data': 'Jay Buhner hit his 38th homer and Edgar Martinez 23rd for Seattle. The Orioles remained tied with the White Sox American League'}


127.0.0.1 - - [06/Apr/2021 03:31:53] "GET / HTTP/1.1" 200 -
